In [ ]:
import csv
import ROOT

In [ ]:
import pandas as pd

In [ ]:
cellstats = pd.read_csv('cellstats.csv')

In [ ]:
cellstats.head()

In [ ]:
cellstats.describe()

In [ ]:
import mplhep
mplhep.style.use('ROOT')
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
np.sum(cellstats.NDF == 0)

In [ ]:
valid = cellstats[cellstats.NDF > 0]

In [ ]:
h = plt.hist(valid.Chi2/valid.NDF, bins='auto')
plt.axvline(5, color='gray', alpha=0.5)
plt.show()

In [ ]:
vals, bins, art = h
bins[np.argmax(vals)]

In [ ]:
chi2ndf = valid.Chi2/valid.NDF
for name, sl in [
    (r'$\chi^2/ndf < 1$', chi2ndf < 1),
    (r'$1 < \chi^2/ndf < 5$', (chi2ndf > 1)&(chi2ndf < 5)),
    (r'$\chi2/ndf > 5$', chi2ndf > 5)
]:
    plt.hist(valid[sl].MPV, range=(0,3), bins=600, label=name, histtype='step')
plt.ylabel('Cells')
plt.xlabel('MPV from Fit / MeV')
plt.yscale('log')
plt.legend()
plt.savefig('chi2ndf.png')
plt.show()


In [ ]:
cellstats["iradius"]=np.sqrt((cellstats["U"]-11)*(cellstats["U"]-11)+(cellstats["V"]-11)*(cellstats["V"]-11))

In [ ]:
picked=cellstats[((cellstats.MPV_Error/cellstats.MPV)<0.05)&(cellstats.Layer<2)&(cellstats.MPV>0)&(cellstats.MPV<1)]

In [ ]:
plt.scatter(picked.MPV,picked.iradius)
#plt.ylabel('Cells')
#plt.xlabel('MPV from Fit / MeV')
#plt.yscale('log')
#plt.legend()
#plt.savefig('chi2ndf.png')
plt.show()

In [ ]:
 y = 1 or 0

In [ ]:
c

In [ ]:
invalid=cellstats[(mpvratio>0.05)&(cellstats.MPV>1)]

In [ ]:
invalid

In [ ]:
mpvratio= abs(cellstats.MPV_Error/cellstats.MPV)

In [ ]:
invalid.describe()n

In [ ]:
negvalid=cellstats[(mpvratio>0.05)&(cellstats.MPV<0)]

In [ ]:
negvalid

In [ ]:
negvalid = negvalid[(negvalid.Layer == 0)]

In [ ]:
negvalid

In [ ]:
invalid = invalid[(invalid.Layer == 0)]

In [ ]:
invalid

In [ ]:
#picked=cellstats[((cellstats.MPV_Error/cellstats.MPV)<0.05)&(cellstats.Layer<2)&(cellstats.MPV>0)&(cellstats.MPV<1)]

In [ ]:
import ROOT
f = ROOT.TFile('hist-no-filtering.root')
d = f.Get('MAC2')
c = ROOT.TCanvas()

In [ ]:
#restricting to the first layer for now
l = 0
for i, row in invalid.iterrows():
    #l = int(row['Layer'])
    u = int(row['U'])
    v =int(row['V'])
    x = 'MAC2_cell_amplitude_l'+ str(l)+'_m0_u'+str(u)+'_v'+str(v)
    print (x)

In [ ]:
x

In [ ]:
h = d.Get(x)

In [ ]:
h.Draw()

In [ ]:
fr = h.Fit('landau','SQM').Get()

In [ ]:
h.Draw()
c.Draw()

In [ ]:
fr.Parameter(1)

In [ ]:
pd

In [ ]:
import pandas as pd
cs = pd.read_csv('cellstats.csv')
cs

In [ ]:
cs[cs.MPV > 1]

In [ ]:
import uproot
import numpy as np
import scipy
import matplotlib.pyplot as plt

# landau is provided by https://github.com/SengerM/landaupy
from landaupy import landau

def scaled_landau(amplitude, constant, mpv, width):
    return constant*landau.pdf(amplitude, x_mpv=mpv, xi=width)

def fit(histogram, drop_zero_bins = True):
    x = histogram.axis().centers()
    y = histogram.values()
    yerr = np.sqrt(histogram.variances())

    if drop_zero_bins:
        x = x[y > 0]
        yerr = yerr[y > 0]
        y = y[y > 0]

    return scipy.optimize.curve_fit(
        scaled_landau,
        xdata = x,
        ydata = y,
        sigma = yerr,
        absolute_sigma = True
    )


def fit_and_plot(histogram, fit_kw = {}, plt_range = None, **plot_kw):
    opt, cov = fit(histogram, **fit_kw)
    if plt_range is None:
        plt_range = histogram.axis().centers()
    mplhep.histplot(histogram, label='Histogram')
    plt.plot(plt_range, scaled_landau(plt_range, *opt), label='Fit', **plot_kw)
    plt.legend()
    return opt, cov

f = uproot.open('hist-no-filtering.root')

In [ ]:
opt, cov = fit_and_plot(
    f['MAC2/MAC2_cell_amplitude_l0_m0_u16_v12'],
    plt_range=np.linspace(0,3,200)
)
plt.axvline(0.26, color='gray')
plt.show()

In [ ]:
print('[Constant x_MPV Sigma]')
print(opt)
print(np.sqrt(np.diag(cov)))

In [ ]:
!pip install git+https://github.com/SengerM/landaupy